In [1]:
import sys
import os

src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
from src.core.llm_generators.specification import AnalyticsSpecGenerator
from src.config.prompts import prompts
from src.config.settings import settings

## Полное описание сразу

In [3]:
user_description = """
Мы хотим построить аналитику для интернет-магазина.
Основная цель — анализировать продажи и поведение покупателей, чтобы повысить выручку и оптимизировать маркетинговые кампании.
Пользователи аналитики — менеджеры по продажам и маркетологи. 
Менеджеры смотрят на продажи и считают выручку. Маркетологи смотрят на активность покупателей и предлагают рекламировать товары на основе спроса.
К процессу также подключен продуктовый аналитик, который помогает маркетологам и менеджерам.

Данные хранятся в PostgreSQL. Есть две основные таблицы:

orders: содержит информацию о заказах (order_id, product_id, timestamp, customer_id, amount)
customers: содержит информацию о клиентах (customer_id, name, region_id, age)

В среднем ежедневно планируется около 20000 заказов. Покупателей сейчас всего около 250000.

Ключевые метрики:

* Общая сумма продаж по дням
* Сумма продаж по регионам
* Количество уникальных покупателей по регионам
* Средний чек

Данные должны обновляться ежедневно, желательно ночью в промежуток между 00:00 и 05:00 (например в 03:00).
В дальнейшем планируется добавить витрины для анализа повторных покупок и сегментации клиентов.
Важно учитывать возможные ограничения по GDPR.
"""

In [3]:
spec_gen = AnalyticsSpecGenerator()

In [ ]:
result = spec_gen.extract_info_from_users_desription(user_description)

## Загрузка уже готового yml

In [6]:
filepath = settings.ARTIFACTS_DIRECTORY / "analytics_spec.yml"
result = spec_gen._from_yml_to_analytics_spec(filepath)
result

AnalyticsSpec(business_process=BusinessProcess(name='Анализ продаж интернет-магазина', description='Анализировать продажи и поведение покупателей для повышения выручки и оптимизации маркетинговых кампаний', schedule='0 3 * * *', roles=[{'role': 'Менеджеры по продажам'}, {'role': 'Маркетологи'}, {'role': 'Продуктовый аналитик'}], goals=['Повышение выручки', 'Оптимизация маркетинговых кампаний'], limitations='Ограничения по GDPR'), data_sources=[DataSource(name='orders', description='Таблица заказов', type='database', data_schema={'order_id': 'int', 'product_id': 'int', 'timestamp': 'timestamp', 'customer_id': 'int', 'amount': 'float'}, database='PostgreSQL', access_method='SQL-запросы', data_volume='20000 заказов в день', limitations=None, recommendations=[], connection_params={}), DataSource(name='customers', description='Таблица клиентов', type='database', data_schema={'customer_id': 'int', 'name': 'varchar', 'region_id': 'int', 'age': 'int'}, database='PostgreSQL', access_method='SQL

## Рекомендации

In [7]:
recommendations = spec_gen.recommendations(business_process=result.business_process,
                                           transformations=result.transformations,
                                           metrics=result.metrics)

In [8]:
recommendations["metrics"]

[{'name': 'Общая сумма продаж по дням',
  'description': 'Сумма продаж за каждый день',
  'calculation_method': 'SUM(amount) по дням',
  'visualization_method': 'График',
  'target_value': None,
  'alerting_rules': None},
 {'name': 'Сумма продаж по регионам',
  'description': 'Сумма продаж по регионам',
  'calculation_method': 'SUM(amount) GROUP BY region_id',
  'visualization_method': 'Диаграмма',
  'target_value': None,
  'alerting_rules': None},
 {'name': 'Количество уникальных покупателей по регионам',
  'description': 'Количество уникальных покупателей по регионам',
  'calculation_method': 'COUNT(DISTINCT customer_id) GROUP BY region_id',
  'visualization_method': 'Таблица',
  'target_value': None,
  'alerting_rules': None},
 {'name': 'Средний чек',
  'description': 'Средний чек по продажам',
  'calculation_method': 'AVG(amount)',
  'visualization_method': 'График',
  'target_value': None,
  'alerting_rules': None},
 {'name': 'Коэффициент конверсии',
  'description': 'Процент посе

In [9]:
recommendations["transformations"]

[]

## Описание по блокам

### 1. Бизнес

- Название бизнес-процесса? e.g. *Аналитика по продажам интернет-магазина*
- Описание целей и стейкхолдеров бизнес-процесса? e.g. *Для увеличения конверсии продаж*
- Как часто нужно обновлять отчеты? e.g. *ежедневно в 15:00, каждый час, еженедельно по вторникам и тд*
- Есть ли ограничения или особенности, которые нужно учесть? e.g. *дедлайны, регламенты, требования по безопасности*

### 2. Источник данных

### 3. Хранилище данных

### 4. Обработка данных

### 5. Метрики